<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase3/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning do Modelo BERT com AmazonTitles-1.3M

Neste notebook, realizaremos o fine-tuning do modelo BERT (`bert-base-uncased`) usando o dataset "The AmazonTitles-1.3MM". O objetivo é treinar o modelo para que ele consiga gerar descrições de produtos com base em seus títulos.

In [1]:
# Configurações e Variáveis Globais

# Configurações de arquivo e diretório
ARQUIVO_TREINAMENTO = 'Datasets/LF-AmazonTitles-1.3M/trn.json.gz'
ARQUIVO_TESTE = 'Datasets/LF-AmazonTitles-1.3M/tst.json.gz'
DIRETORIO_CHECKPOINTS_LOCAL = './checkpoints'
DIRETORIO_CHECKPOINTS_COLAB = '/content/drive/MyDrive/FIAP/1IADT/Fase-3/tc/checkpoints'
MODELO_FINAL = './FIAP-1IADT-Grupo28'

# Configurações de modelo
MODELO_BASE = 'bert-base-uncased'
NUM_LABELS = 2

# Configurações de treinamento
TAMANHO_BLOCO = 10000
MAX_LENGTH = 128
NUM_TRAIN_EPOCHS = 3
PER_DEVICE_TRAIN_BATCH_SIZE = 16
SAVE_STEPS = 1000
SAVE_TOTAL_LIMIT = 2

# Configurações de output
OUTPUT_DIR = './results'
LOGGING_DIR = './logs'

### 1. Instalar dependências


In [2]:
# Instalar as bibliotecas necessárias
%pip install datasets transformers torch pandas 'transformers[torch]' gdown
%pip install --upgrade jupyter ipywidgets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [3]:
import sys
import subprocess

def install_and_import(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package])

# Install required packages
packages = ['jupyter', 'ipywidgets', 'tqdm']
for package in packages:
    install_and_import(package)

# Restart the kernel after running this cell
print("Please restart the Jupyter kernel to apply changes.")

Please restart the Jupyter kernel to apply changes.


## 2. Importar as Bibliotecas e preparar o Ambiente

In [4]:
import os
import torch
import gzip
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import json
import sys

if 'google.colab' in sys.modules:
    install_and_import('google-auth-oauthlib')

# Verificar se temos acesso a uma GPU no Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## 3. Carregar o Tokenizer e o Modelo BERT

In [5]:
# Carregar o tokenizer BERT
tokenizer = BertTokenizer.from_pretrained(MODELO_BASE)

# Carregar o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained(MODELO_BASE, num_labels=NUM_LABELS)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 4. Classe Dataset para Gerenciamento de Dados

In [6]:
class AmazonTitlesDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


## 5. Leitura em Chunks e Tokenização (para JSONL compactado em GZIP)
Vamos ajustar a função de leitura para processar arquivos JSONL. Cada linha é um objeto JSON separado, então a função simplesmente lê uma linha por vez e cria blocos (chunks).

In [7]:
def ler_arquivo_em_blocos_jsonl_gz(caminho_arquivo, tamanho_bloco=10000):
    with gzip.open(caminho_arquivo, 'rt') as f:  # 'rt' para ler como texto
        bloco = []
        for i, linha in enumerate(f):
            bloco.append(json.loads(linha.strip()))  # Lê uma linha como JSON
            if (i + 1) % tamanho_bloco == 0:
                yield bloco
                bloco = []
        if bloco:
            yield bloco

def processar_e_tokenizar_chunk(chunk, max_length=128):
    titles = [item['title'] for item in chunk]
    descriptions = [item['content'] for item in chunk]

    # Concatenar título e descrição
    inputs = [f"{title} [SEP] {description}" for title, description in zip(titles, descriptions)]

    # Tokenização
    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length)

    # Exemplo de rótulos fictícios; substitua conforme necessário
    labels = [1] * len(chunk)

    return encodings, labels

# Função para detectar o ambiente de execução
def detectar_ambiente():
    if 'google.colab' in sys.modules:
        return 'colab_gpu' if torch.cuda.is_available() else 'colab_cpu'
    else:
        return 'local'

# Função para obter o diretório de checkpoints
def obter_diretorio_checkpoints():
    ambiente = detectar_ambiente()
    if ambiente.startswith('colab'):
        try:
            from google.colab import drive # type: ignore
            drive.mount('/content/drive')
            return DIRETORIO_CHECKPOINTS_COLAB
        except ImportError:
            print("Não foi possível importar o módulo 'drive'. Verifique se está no ambiente Colab.")
            return None
    else:
        return './checkpoints'

def carregar_ultimo_checkpoint():
    checkpoint_dir = obter_diretorio_checkpoints()

    # Procurar por checkpoints existentes
    checkpoints = sorted(
        [c for c in os.listdir(checkpoint_dir) if c.startswith('checkpoint_')],
        key=lambda x: int(x.split('_')[-1])  # Extrai o número do checkpoint e ordena numericamente
    )

    if checkpoints:
        # Pega o último checkpoint
        ultimo_checkpoint = checkpoints[-1]
        ultimo_chunk = int(ultimo_checkpoint.split('_')[-1])

        # Carrega o modelo e tokenizer do último checkpoint
        modelo = BertForSequenceClassification.from_pretrained(f'{checkpoint_dir}/{ultimo_checkpoint}')
        tokenizer = BertTokenizer.from_pretrained(f'{checkpoint_dir}/{ultimo_checkpoint}')

        return modelo, tokenizer, ultimo_chunk

    return None, None, -1  # Retorna -1 para indicar que nenhum chunk foi processado ainda

def salvar_progresso(model, tokenizer, chunk_idx):
    checkpoint_dir = obter_diretorio_checkpoints()

    # Salva o modelo e tokenizer no checkpoint atual
    model.save_pretrained(f'{checkpoint_dir}/checkpoint_{chunk_idx}')
    tokenizer.save_pretrained(f'{checkpoint_dir}/checkpoint_{chunk_idx}')

    # Salva o progresso do chunk processado
    with open(f'{checkpoint_dir}/ultimo_chunk.txt', 'w') as f:
        f.write(str(chunk_idx))

    checkpoints = sorted(
        [c for c in os.listdir(checkpoint_dir) if c.startswith('checkpoint_')],
        key=lambda x: int(x.split('_')[-1])  # Extrai o número do nome e ordena numericamente
    )

    for checkpoint in checkpoints[:-2]:
        os.system(f'rm -rf {checkpoint_dir}/{checkpoint}')

## 6. Configuração do Treinamento


In [8]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,                                   # Diretório de saída para os resultados
    num_train_epochs=NUM_TRAIN_EPOCHS,                       # Número de épocas
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE, # Tamanho do batch
    save_steps=SAVE_STEPS,                                   # Salvar checkpoints a cada 1000 passos
    save_total_limit=SAVE_TOTAL_LIMIT,                       # Limite de dois checkpoints salvos
    logging_dir=LOGGING_DIR,                                 # Diretório de logs
)


## 7. Função de Treinamento por Chunk


In [9]:
def treinar_com_chunk(encodings, labels):
    dataset = AmazonTitlesDataset(encodings, labels)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )
    # Treina o modelo usando o chunk atual
    trainer.train()


## 8. Baixar dados o Google Drive

Vamos baixar os dados do Google Drive para acessar os arquivos que contêm os dados de treinamento e teste.

In [10]:
!mkdir -p Datasets
!cd Datasets; gdown 12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK;
!cd Datasets; unzip LF-Amazon-1.3M.raw.zip; mkdir -p LF-AmazonTitles-1.3M/raw; mv LF-Amazon-1.3M/* LF-AmazonTitles-1.3M; rmdir LF-Amazon-1.3M

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=0a563005-099d-46e7-93b9-a95d6a79fed3
To: /content/Datasets/LF-Amazon-1.3M.raw.zip
100% 890M/890M [00:13<00:00, 64.5MB/s]
Archive:  LF-Amazon-1.3M.raw.zip
   creating: LF-Amazon-1.3M/
  inflating: LF-Amazon-1.3M/lbl.json.gz  
  inflating: LF-Amazon-1.3M/trn.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_test.txt  
  inflating: LF-Amazon-1.3M/tst.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_train.txt  


## 9. Processar e Treinar em Chunks
A função principal que faz a leitura do arquivo JSONL em chunks e realiza o fine-tuning do modelo BERT em cada chunk.

In [ ]:
# Processar o arquivo compactado trn.json.gz e realizar o fine-tuning em chunks
caminho_arquivo = ARQUIVO_TREINAMENTO

ambiente = detectar_ambiente()
print(f"Ambiente detectado: {ambiente}")

# Carregar o último checkpoint, se existir
modelo_carregado, tokenizer_carregado, ultimo_chunk_processado = carregar_ultimo_checkpoint()
if modelo_carregado is not None:
    model = modelo_carregado
    tokenizer = tokenizer_carregado
    print(f"Retomando o treinamento a partir do chunk {ultimo_chunk_processado + 1}")
else:
    # Inicializar modelo e tokenizer se não houver checkpoint
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    ultimo_chunk_processado = -1  # Nenhum chunk processado ainda

# Processar chunks
for i, chunk in enumerate(ler_arquivo_em_blocos_jsonl_gz(caminho_arquivo, tamanho_bloco=TAMANHO_BLOCO), start=1):
    # Pular chunks estritamente menores que o último chunk processado
    if i <= ultimo_chunk_processado:
        print(f"Chunk {i} já processado. Pulando...")
        continue

    print(f"Processando chunk {i}")

    encodings, labels = processar_e_tokenizar_chunk(chunk)

    # Executar treinamento no chunk atual
    treinar_com_chunk(encodings, labels)

    # Salvar o progresso após cada chunk
    salvar_progresso(model, tokenizer, i)

    print(f"Treinamento com chunk {i} completo.")

Ambiente detectado: colab_gpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Retomando o treinamento a partir do chunk 52
Chunk 1 já processado. Pulando...
Chunk 2 já processado. Pulando...
Chunk 3 já processado. Pulando...
Chunk 4 já processado. Pulando...
Chunk 5 já processado. Pulando...
Chunk 6 já processado. Pulando...
Chunk 7 já processado. Pulando...
Chunk 8 já processado. Pulando...
Chunk 9 já processado. Pulando...
Chunk 10 já processado. Pulando...
Chunk 11 já processado. Pulando...
Chunk 12 já processado. Pulando...
Chunk 13 já processado. Pulando...
Chunk 14 já processado. Pulando...
Chunk 15 já processado. Pulando...
Chunk 16 já processado. Pulando...
Chunk 17 já processado. Pulando...
Chunk 18 já processado. Pulando...
Chunk 19 já processado. Pulando...
Chunk 20 já processado. Pulando...
Chunk 21 já processado. Pulando...
Chunk 22 já processado. Pulando...
Chunk 23 já processado. Pulando...


Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Treinamento com chunk 52 completo.
Processando chunk 53


Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Treinamento com chunk 53 completo.
Processando chunk 54


...

Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Treinamento com chunk 56 completo.
Processando chunk 57


Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Treinamento com chunk 57 completo.
Processando chunk 58


Step,Training Loss


In [11]:
# Processar o arquivo compactado trn.json.gz e realizar o fine-tuning em chunks
caminho_arquivo = ARQUIVO_TREINAMENTO

ambiente = detectar_ambiente()
print(f"Ambiente detectado: {ambiente}")

i = 1

# Processar chunks
for i, chunk in enumerate(ler_arquivo_em_blocos_jsonl_gz(caminho_arquivo, tamanho_bloco=TAMANHO_BLOCO), start=1):
  continue


print(f"Total de chunks: {i}")

Ambiente detectado: colab_cpu
Total de chunks: 225


## 10. Salvar o Modelo Fine-Tuned
Depois de processar todos os chunks e realizar o fine-tuning do modelo, salvamos o modelo treinado.

In [ ]:
# Salvar o modelo fine-tuned e o tokenizer
model.save_pretrained(MODELO_FINAL)
tokenizer.save_pretrained(MODELO_FINAL)

print("Modelo fine-tuned salvo com sucesso.")

## 11. Validação do Modelo Fine-Tuned

In [ ]:
# Carregar o modelo fine-tuned
model = BertForSequenceClassification.from_pretrained(MODELO_FINAL)
tokenizer = BertTokenizer.from_pretrained(MODELO_FINAL)
model.to(device)


In [ ]:
# Função de avaliação
def avaliar_modelo(model, dataloader):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    return accuracy


In [ ]:
# Executar a avaliação
total_accuracy = 0
total_batches = 0

for chunk in ler_arquivo_em_blocos_jsonl_gz(ARQUIVO_TESTE):
    encodings, labels = processar_e_tokenizar_chunk(chunk)
    dataset = AmazonTitlesDataset(encodings, labels)
    dataloader = DataLoader(dataset, batch_size=32)

    accuracy = avaliar_modelo(model, dataloader)
    total_accuracy += accuracy
    total_batches += 1

    print(f"Acurácia do batch: {accuracy:.4f}")

average_accuracy = total_accuracy / total_batches
print(f"Acurácia média: {average_accuracy:.4f}")